In [3]:
from google.cloud import bigquery
from google.oauth2 import service_account
import os 
from dotenv import load_dotenv 
import pandas as pd
from datetime import datetime, timedelta

load_dotenv ()

PROJECT_ID = os.getenv("PROJECT_ID")


In [ ]:
credentials = service_account.Credentials.from_service_account_file(
    filename='credentials/credentials.json'
)
client = bigquery.Client(credentials=credentials, project=PROJECT_ID)

# Process day by day to avoid the response size limit
start_date = datetime(2025, 10, 1)
end_date = datetime(2025, 10, 7)
current_date = start_date

all_dfs = []

while current_date <= end_date:
    date_str = current_date.strftime('%Y-%m-%d')
    
    QUERY_TX = f""" 
    SELECT * 
    FROM `nodely-data-pub.algorand_mainnet_daily.transactions`
    WHERE DATE(block_timestamp) = '{date_str}'
    """
    
    try:
        query_job = client.query(QUERY_TX)
        df_day = query_job.to_dataframe()  
        all_dfs.append(df_day)
        print(f"Loaded {date_str}: {len(df_day)} rows")
        
    except Exception as e:
        print(f"Error loading {date_str}: {e}")
    
    current_date += timedelta(days=1)

# Combine all days into one DataFrame
df = pd.concat(all_dfs, ignore_index=True)
print(f"Total rows loaded: {len(df)}")